In [5]:
import pandas as pd
from Project import collector
import sys

In [6]:
std_origin = sys.stdout
sys.stdout = open("ComDump", "w", encoding="utf-8")
data = collector.single_run_collector(with_communication=True, seed=123)
#sys.stdout.flush()
sys.stdout = std_origin

In [8]:
import re
#sys.stdout = std_origin
with open("ComDump", "r", encoding="utf-8") as file:
    lines = file.readlines()
    succed = []
    for i in range(len(lines)):
        if re.search("(The request is completed)|(The negotiation has reached an agreement)", lines[i]) is not None:
            succed.append(i)
    print(len(succed))

357
